# INFO

This notebook will be used for construction and testing purposes while designing model within Kedro framework. 

In [1]:
##############################################################################
# It is recommended to create new virtual environment for each Kedro project #
##############################################################################

# Uncomment and run the line below if your environment does't have
# Kedro or any other dependencies needed.

#! pip install -r requirements.txt
%load_ext kedro.ipython

[11/05/24 17:37:03] INFO     Using 'd:\Program Files\Python\Python                                  __init__.py:270
                             Venv\lc\lib\site-packages\kedro\framework\project\rich_logging.yml' as                
                             logging configuration.                                                                

                    INFO     Registered line magic '%reload_kedro'                                   __init__.py:61

                    INFO     Registered line magic '%load_node'                                      __init__.py:63

                    INFO     Resolved project path as: c:\Users\chemerys\Downloads\lending-club.    __init__.py:178
                             To set a different path, run '%reload_kedro <project_root>'                           

[11/05/24 17:37:07] INFO     Kedro project lending_club                                             __init__.py:144

                    INFO     Defined global variable 'context', 'session', 'catalog' and            __init__.py:145
                             'pipelines'                                                                           

[11/05/24 17:37:09] INFO     Registered line magic 'run_viz'                                        __init__.py:151


----

# Baseline model

As the dataset needs transformation like imputation and normalization, for avoiding data leakage, all transformations will be done within model pipeline and fitting only on training data on model fitting stage. So I'm going to split initial typed dataset to train/test sets and balance train set, ignoring all previously transformed datasets.

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.compose import make_column_transformer
# from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import pandas as pd
import logging
from lending_club.pipelines.analysis.nodes import features_eng
from lending_club.pipelines.encode.nodes import _default_status
from imblearn.pipeline import make_pipeline

logger = logging.getLogger(__name__)

def split_dataset(df: pd.DataFrame, params: dict):
    y = _default_status(df, params)
    X = df #TODO Remove .drop('default_status', axis=1)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=params['test_size'],
        random_state=params['random_state']
    )
    return X_train, X_test, y_train, y_test


def train_model(X_train, y_train, regressor, params: dict):
    try:
        regressor.set_params(**params['fit_options']).fit(X_train, y_train) 
    except:
        regressor.fit(X_train, y_train)
    return regressor

def evaluate_metrics(model: object, params: dict, X_true: object, y_true: object) -> pd.DataFrame:
    y_pred = model.predict(X_true)
    # NN model return probabilities that we should translate to
    # True/False based on treshold = 0.5
    if not isinstance(y_pred, list):
        y_pred = (y_pred > 0.5)
    metrics = pd.DataFrame()
    metrics['accuracy']  = {params['name']: accuracy_score(y_true, y_pred)}
    metrics['precision'] = {params['name']: precision_score(y_true, y_pred)}
    metrics['recall']    = {params['name']: recall_score(y_true, y_pred)}
    metrics['f1']        = {params['name']: f1_score(y_true, y_pred)}
    metrics['roc_auc']   = {params['name']: roc_auc_score(y_true, y_pred)}
    metrics['conf_mtx']  = {params['name']: confusion_matrix(y_true, y_pred)}
    print("Model's scores:\n", metrics)



'''
Merging in one pipeline functions from:
    - data_process pipeline
    - data_clean pipeline
    - encode pipeline
'''

def _drop_duplicates(df: pd.DataFrame):
    return df.drop_duplicates()

# Function to encode emp_length to number
def _parse_emp_len(df: pd.DataFrame, emp_len: str) -> pd.DataFrame:
    df[emp_len] = df[emp_len].str.split(" ").str[0].str.replace("+", "").str.replace("<", "0").astype(int)
    return df[[emp_len]]

def model_pipeline(params: dict):
    # Drop duplicates and selected features
    drop_trash = make_column_transformer(
        ('drop', params["drop_list"]),
        remainder=FunctionTransformer(_drop_duplicates)
    )
    # pipelines/data_clean replacement
    data_clean = make_column_transformer(
        (SimpleImputer(missing_values=None, strategy='constant', fill_value='none'), params['none_list']),
        (SimpleImputer(missing_values=None, strategy='most_frequent'), params['freq_list']),
        (SimpleImputer(strategy='constant', fill_value=0), params['fill_zero']),
        (SimpleImputer(strategy='median'), params['fill_med']),
        remainder='passthrough'
    )
    # pipelines/encode replacement
    encode = make_column_transformer(
        (OrdinalEncoder(), params['category']),
        (FunctionTransformer(_parse_emp_len), params['emp_len']),
        remainder='passthrough'
    )
    # feature engineering node replacement
    new_features = make_column_transformer(
        (FunctionTransformer(features_eng), []),
        remainder='passthrough'
    )
    # scale features selected for model and drop the rest
    normalizer = make_column_transformer(
        (StandardScaler(), params['model_features']),
        remainder='drop'
    )
    regressor_params = params['model_options']['regressor_options']
    pipeline = make_pipeline(
        drop_trash,
        data_clean,
        encode,
        new_features,
        normalizer,
        SMOTE(random_state=params['random_state']),
        # CatBoostClassifier(
        #     iterations=regressor_params['iterations'],						# Maximum number of boosting iterations
        #     learning_rate=regressor_params['learning_rate'],					# Learning rate
        #     eval_metric=regressor_params['eval_metric'],					    # Metric to monitor
        #     custom_loss=regressor_params['custom_loss'],                      # Additional metrics to plot
        #     early_stopping_rounds=regressor_params['early_stopping_rounds'],	# Stop if no improvement after X iterations
        #     od_type=regressor_params['od_type'],						        # Overfitting detection type (detect after fixed number of non-improving iterations)
        #     random_seed=regressor_params['random_seed'],
        #     verbose=regressor_params['verbose'],						        # Print log every X iterations
        #     eval_fraction=regressor_params['eval_fraction']                   # Fraction of training dataset for validation
        # )
        RandomForestClassifier(regressor_params)
    )
    return pipeline

# Load parameters
params = catalog.load("parameters")
params['model_features'] = catalog.load("params:model_features")
params['model_options'] = catalog.load("params:model_options")

# Construct pipeline
model = model_pipeline(params)
model

[05.11.2024 17:56:37] INFO     Loading data from parameters (MemoryDataset)...                  data_catalog.py:389

                      INFO     Loading data from params:model_features (MemoryDataset)...       data_catalog.py:389

                      INFO     Loading data from params:model_options (MemoryDataset)...        data_catalog.py:389

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder=FunctionTransformer(func=<function _drop_duplicates at 0x0000027951B44DC0>),
                                   transformers=[('drop', 'drop',
                                                  ['Unnamed: 0.1', 'Unnamed: 0',
                                                   'url', 'emp_title', 'title',
                                                   'zip_code'])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('simpleimputer-1',
                                                  SimpleImputer(...
                                                   'home_ownership',
                                                   'max_bal_bc',
                                                   'mths_since_recent_inq',
                                                   'open_rv_24m',
                                                   'verification_status',
                                                   'open_acc_6m', 'sub_grade',
                                                   'bc_open_to_buy',
                                                   'mo_sin_rcnt_rev_tl_op',
                                                   'inq_last_6mths_adj',
                                                   'tot_cur_bal',
                                                   'mort_acc_adj', 'term',
                                                   'dti_adj'])])),
                ('smote', SMOTE(random_state=42)),
                ('randomforestclassifier',
                 RandomForestClassifier(n_estimators={'n_estimators': 100,
                                                      'random_state': 42}))])

In [12]:
X_train, X_test, y_train, y_test = split_dataset(
    catalog.load("intermediate_lc_dataset"),
    params)


[05.11.2024 17:57:00] INFO     Loading data from intermediate_lc_dataset (ParquetDataset)...    data_catalog.py:389

In [13]:
train_model(X_train, y_train, model, params['model_options'])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in train_model:34                                                                                │
│                                                                                                  │
│    31                                                                                            │
│    32 def train_model(X_train, y_train, regressor, params: dict):                                │
│    33 │   try:                                                                                   │
│ ❱  34 │   │   regressor.set_params(**params['fit_options']).fit(X_train, y_train)                │
│    35 │   except:                                                                                │
│    36 │   │   regressor.fit(X_train, y_train)                                                    │
│    37 │   return regressor                                                                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'fit_options'

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ d:\Program Files\Python\Python Venv\lc\lib\site-packages\sklearn\utils\_indexing.py:338 in       │
│ _get_column_indices                                                                              │
│                                                                                                  │
│   335 │   │   return _get_column_indices_for_bool_or_int(key, n_columns)                         │
│   336 │   else:                                                                                  │
│   337 │   │   try:                                                                               │
│ ❱ 338 │   │   │   all_columns = X.columns                                                        │
│   339 │   │   except AttributeError:                                                             │
│   340 │   │   │   raise ValueError(                                                              │
│   341 │   │   │   │   "Specifying the columns using strings is only supported for dataframes."   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'numpy.ndarray' object has no attribute 'columns'

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 train_model(X_train, y_train, model, params['model_options'])                                │
│   2                                                                                              │
│                                                                                                  │
│ in train_model:36                                                                                │
│                                                                                                  │
│    33 │   try:                                                                                   │
│    34 │   │   regressor.set_params(**params['fit_options']).fit(X_train, y_train)                │
│    35 │   except:                                                                                │
│ ❱  36 │   │   regressor.fit(X_train, y_train)                                                    │
│    37 │   return regressor                                                                       │
│    38                                                                                            │
│    39 def evaluate_metrics(mode